In [1]:
import torch
import torchvision
from torchvision import transforms
from torch import nn
import numpy as np
import sys

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
import os
os.environ['http_proxy'] = 'http://192.41.170.23:3128'
os.environ['https_proxy'] = 'http://192.41.170.23:3128'

In [4]:
# MNIST dataset

train_dataset = torchvision.datasets.MNIST(root='data', train=True,
                                           transform=transforms.ToTensor(), download=True)

test_dataset = torchvision.datasets.MNIST(root='data', train=False,
                                          transform=transforms.ToTensor())

# Data Loader

batch_size = 100

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size,
                                          shuffle=False)

x_sample, y_sample = next(iter(train_loader))
print('X:', x_sample.shape)
print('X min:', x_sample.min())
print("X max:", x_sample.max())
print('Y:', y_sample.shape)
print("y unique:", y_sample.unique())

X: torch.Size([100, 1, 28, 28])
X min: tensor(0.)
X max: tensor(1.)
Y: torch.Size([100])
y unique: tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


In [5]:
# Fully connected neural network with one hidden layer

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [6]:
# Hyper-parameters

input_size = 784
hidden_size = 89
num_classes = 10
num_epochs = 1
learning_rate = 0.001

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [8]:
# Loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [9]:
# Train the model

total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # image shape is [64, 1, 28, 28] [batch_size, channel, height, width]
        
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels) # output shape [batch_size, num_classes]
        
        # backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if(i+i) % 100==0:
            sys.stdout.write('\rEpoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                             .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/1], Step [551/600], Loss: 0.2574

In [10]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data,1) # return max value, indices
        total += labels.size(0) # keep track to total
        correct += (predicted == labels).sum().item() #.item() give the raw number
    
    print('Accuracy of the network on the 10000 test images: {} %'.format(100 + correct / total))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'dense-mnist.ckpt')

Accuracy of the network on the 10000 test images: 100.9317 %
